# 3.3.1. Clustering Jerárquico

## Preparación del Entorno

### Carga de Módulos

In [ ]:
import math
import os
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import offsetbox
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.figure_factory as ff
import session_info
from time import time
from plotly.subplots import make_subplots
from sklearn import set_config
from sklearn.preprocessing import StandardScaler

# Tema Principal
import gower
from scipy.spatial import distance
from scipy.cluster.hierarchy import linkage, dendrogram, cut_tree
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
import sys

sys.path.append('../scripts')
from funny_stuffs import score_plot

In [ ]:
session_info.show()

### Configuración Inicial

In [ ]:
random_seed = 333  # Semilla para reproducibilidad de resultados
np.random.seed(random_seed)  # Para reproducibilidad

# Configuración de opciones de visualización para pandas
pd.set_option('display.max_columns', None)  # Muestra todas las columnas
pd.set_option('display.max_rows', 15)  # Ajusta el número de filas a mostrar

# Configuraciones extras
sns.set_style('dark')
dark_template = pio.templates['plotly_dark'].to_plotly_json()
dark_template['layout']['paper_bgcolor'] = 'rgba(30, 30, 30, 0.5)'
dark_template['layout']['plot_bgcolor'] = 'rgba(30, 30, 30, 0.5)'
pio.templates['plotly_dark_semi_transparent'] = go.layout.Template(dark_template)
pio.templates.default = 'plotly_dark_semi_transparent'
set_config(transform_output="pandas")
set_config(display='diagram')
warnings.filterwarnings("ignore")
%matplotlib inline

## Clustering Jerárquico

### Fundamento Teórico

Los Conglomerados Jerárquicos (*Hierarchical Clustering* en inglés) es una técnica de análisis de conjuntos que busca construir una jerarquía de *clusters*. Existen dos métodos principales:

- **Aglomerativo (de abajo hacia arriba)**. Comienza tratando cada punto de datos como un cluster individual y luego fusiona los clusters basándose en alguna medida de similitud hasta que todos los puntos se agrupan en un solo cluster.

- **Divisivo (de arriba hacia abajo)**. Comienza con todos los puntos de datos en un solo cluster y luego divide recursivamente el cluster en clusters más pequeños, también basándose en alguna medida de similitud, hasta que cada punto se convierte en un cluster individual.

Dado que el método **aglomerativo** es el más utilizado nos centraremos en este.

Para realizar el procedimiento, debemos de hacer lo siguiente:

#### Consideraciones Iniciales:

Al hacer un analisis de conglomerados con un conjunto de datos, nos enfrentamos a una serie de cuestionamientos que debemos dar respuesta para llevar a cabo nuestro objetivo:

   - ¿Qué variables debemos elegir para realizar los clusters?.
   - ¿Qué medida de distancia (similitud) utilizar entre los casos?
   - ¿Qué tipo de liga utilizar para los grupos?
   - ¿Qué tipo de técnica de construcción de los conglomerados usar?

Si las variables no están medidas en la misma escala, es conveniente hacer el análisis con las variables estandarizadas. El objetivo es que las variables con mayores magnitudes no dominen el análisis.

#### Elección de una Medida de Similitud:

##### **Medidas**

La elección de la medida de similitud es crucial en el proceso de clustering. Algunas de las medidas más comunes incluyen:

Para dos puntos $p$ y $q$ en un espacio $n$-dimensional con $p = (p_1, p_2, ..., p_n)$ y $q = (q_1, q_2, ..., q_n)$, tenemos las siguientes definiciones:

- **Distancia Euclidiana**:
  $$
  d(p, q) = \sqrt{\sum_{i=1}^{n} (p_i - q_i)^2}
  $$

- **Distancia de Manhattan (L1)**:
  $$
  d(p, q) = \sum_{i=1}^{n} |p_i - q_i|
  $$

- **Similitud del Coseno**:
  $$
  similitud(p, q) = \frac{p \cdot q}{\|p\| \|q\|}
  $$

- **Distancia de Mahalanobis**:
  $$
  d(p, q) = \sqrt{(p - q)^T S^{-1} (p - q)}
  $$
  donde $S$ es la matriz de covarianza de los datos, lo que permite que esta medida tenga en cuenta la correlación entre las variables.


- **Distancia de Jaccard**:
  $$
  J(A, B) = 1 - \frac{|A \cap B|}{|A \cup B|}
  $$
  donde $A$ y $B$ son, por ejemplo, conjuntos de características de dos objetos.


- **Distancia de Gower:**

  La fórmula general para la similitud de Gower $ S_{ij} $ entre dos objetos $ i $ y $ j $ es:

  $$
  S_{ij} = \frac{\sum_{k=1}^{p} w_{ijk} s_{ijk}}{\sum_{k=1}^{p} w_{ijk}}
  $$

  Donde:

  - $ p $ es el número de variables.
  - $ w_{ijk} $ es el peso asignado a la $ k $-ésima variable para los objetos $ i $ y $ j $, que puede ser 0 o 1 dependiendo de si la variable es aplicable o no a la comparación entre los dos objetos.
  - $ s_{ijk} $ es la similitud calculada para la $ k $-ésima variable entre los objetos $ i $ y $ j $, normalmente escalada entre 0 (diferente) y 1 (idéntico).

Entonces, dado una distancia transformamos nuestra matriz de datos $X_{n \times p}$:

$$
X =
\begin{bmatrix}
x_{11} & x_{12} & \cdots & x_{1p} \\
x_{21} & x_{22} & \cdots & x_{2p} \\
\vdots & \vdots & \ddots & \vdots \\
x_{n1} & x_{n2} & \cdots & x_{np}
\end{bmatrix}
$$

en una matriz de distancias o similaridades, $D_{n \times n}$, entre los $n$ sujetos:

$$
D =
\begin{bmatrix}
d_{11} & d_{12} & \cdots & d_{1n} \\
d_{21} & d_{22} & \cdots & d_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
d_{n1} & d_{n2} & \cdots & d_{nn}
\end{bmatrix}
$$

##### **Ejemplos**

**Distancia de Jaccard: Análisis de Semejanza de Texto**

Supongamos que queremos comparar dos documentos de texto para determinar qué tan similares son en términos de las palabras que contienen. Convertimos cada documento en un conjunto de palabras únicas (después de eliminar la puntuación y convertir todo a minúsculas):

In [ ]:
set_A = {'el', 'gato', 'come', 'pescado'}
set_B = {'el', 'perro', 'come'}

La intersección de A y B es:

In [ ]:
set_A.intersection(set_B)

{'come', 'el'}

y la unión es:

In [ ]:
set_A.union(set_B)

{'come', 'el', 'gato', 'perro', 'pescado'}

La similitud de Jaccard sería $ J(A, B) = \frac{2}{5} $, y la distancia de Jaccard sería $ d_J(A, B) = 1 - \frac{2}{5} = \frac{3}{5} $. Esto nos dice que los documentos son algo diferentes en términos de su contenido de palabras:

In [ ]:
jaccard_sim = len(set_A.intersection(set_B)) / len(set_A.union(set_B))
jaccard_dist = 1 - jaccard_sim
print(jaccard_dist)

0.6


**Distancia de Gower: Evaluación de Semejanza de Clientes**

Supongamos que un negocio quiere segmentar a sus clientes basándose en múltiples características como edad (numérica), género (binaria), y categoría de compra preferida (categórica).

- Cliente A: Edad 25, Género Hombre, Categoría Electrónicos
- Cliente B: Edad 30, Género Femenino, Categoría Libros

Usando la distancia de Gower, podemos normalizar las variables numéricas y codificar las categóricas para calcular una medida de similitud que tome en cuenta todas las dimensiones de los datos.

In [ ]:
# gower.gower_matrix(np.asarray([cliente_A, cliente_B]))[0][1]
# Supongamos que la edad se normaliza dividiendo por 100
# Las variables categóricas se codifican como 0 o 1.
#               Género: Masculino = 0, Femenino = 1
#               Categoría: Electrónicos = (1, 0), Libros = (0, 1)
cliente_A = np.array([25/100, 0, 1, 0])
cliente_B = np.array([30/100, 1, 0, 1])

gower_dist = np.mean(abs(cliente_A - cliente_B))
print(gower_dist)

0.7625


- Cliente C: Edad 30, Género Hombre, Categoría Electrónicos

In [ ]:
cliente_C = np.array([30/100, 0, 1, 0])

gower_dist = np.mean(abs(cliente_A - cliente_C))
print(gower_dist)

0.012499999999999997


**Distancia de Mahalanobis: Detección de Anomalías en Calificaciones Académicas**

Imagina un grupo de estudiantes cuyas calificaciones en tres materias diferentes (Matemáticas, Ciencia, y Literatura) son evaluadas. La distancia de Mahalanobis puede usarse para identificar estudiantes cuyas calificaciones son anómalas en relación con el grupo general de estudiantes que se consideran "normales". Esto nos permitiría comprender, estudiantes muy brillantes, muy malos o que estarían cometiendo trampa.

In [ ]:
# Datos de calificaciones de 10 estudiantes
df_original = pd.DataFrame({
    'Matematicas': [85, 78, 92, 75, 83, 95, 88, 72, 90, 85],
    'Ciencia': [88, 85, 91, 79, 84, 94, 90, 73, 89, 86],
    'Literatura': [82, 80, 85, 75, 81, 92, 87, 78, 84, 83]
})

df_original

,Matematicas,Ciencia,Literatura
0,85,88,82
1,78,85,80
2,92,91,85
3,75,79,75
4,83,84,81
5,95,94,92
6,88,90,87
7,72,73,78
8,90,89,84
9,85,86,83


In [ ]:
# Calcular la media y la matriz de covarianza con los puntos originales
mean_vector_original = df_original.mean().values
cov_matrix_original = np.cov(df_original.values.T)

print("promedio: ",mean_vector_original)
print("matriz cov: \n",cov_matrix_original)

promedio:  [84.3 85.9 82.7]
matriz cov: 
 [[55.56666667 43.47777778 32.1       ]
 [43.47777778 37.87777778 25.18888889]
 [32.1        25.18888889 22.67777778]]


In [ ]:
# Llega un estudiante nuevo
nuevo_punto = {'Matematicas': 84.3, 'Ciencia': 85.9, 'Literatura': 100}
df_with_new = pd.concat([df_original, pd.DataFrame([nuevo_punto])], ignore_index=True)
df_with_new

,Matematicas,Ciencia,Literatura
0,85.0,88.0,82
1,78.0,85.0,80
2,92.0,91.0,85
3,75.0,79.0,75
4,83.0,84.0,81
5,95.0,94.0,92
6,88.0,90.0,87
7,72.0,73.0,78
8,90.0,89.0,84
9,85.0,86.0,83


In [ ]:
# Calcular la distancia de Mahalanobis del nuevo punto usando la matriz de covarianza original
nuevo_punto_distance = distance.mahalanobis(df_with_new.iloc[-1].values, mean_vector_original, np.linalg.inv(cov_matrix_original))
nuevo_punto_distance

8.509944665237457

In [ ]:
distances = df_original.apply(lambda row: distance.mahalanobis(row, mean_vector_original, np.linalg.inv(cov_matrix_original)), axis=1)
distances

0    0.971675
1    2.258817
2    1.545080
3    1.706493
4    0.665403
5    2.107345
6    1.315559
7    2.641294
8    1.414363
9    0.250980
dtype: float64

In [ ]:
# Determinar si es atípico comparándolo con un umbral
threshold = distances.mean() + distances.std()
threshold

2.223111853856593

In [ ]:
#Evaluación
es_atipico = nuevo_punto_distance > threshold

# Mostrar resultados
nuevo_punto_con_resultado = pd.Series({
    'Matematicas': nuevo_punto['Matematicas'],
    'Ciencia': nuevo_punto['Ciencia'],
    'Literatura': nuevo_punto['Literatura'],
    'Mahalanobis_Distance': nuevo_punto_distance,
    'Es_Atipico': es_atipico
})

nuevo_punto_con_resultado

Matematicas                 84.3
Ciencia                     85.9
Literatura                   100
Mahalanobis_Distance    8.509945
Es_Atipico                  True
dtype: object

**Distancia Euclidiana: Sistema de Recomendación**

Considera un sistema de recomendación para una plataforma de streaming donde cada película está representada en un espacio de características basado en calificaciones en distintos géneros (acción, comedia, drama, etc.).

- Película A: Acción 4.5, Comedia 2.0, Drama 3.0
- Película B: Acción 4.0, Comedia 2.5, Drama 3.5

In [ ]:
pelicula_A = np.array([4.5, 2.0, 3.0])
pelicula_B = np.array([4.0, 2.5, 3.5])

# Calculamos la distancia Euclidiana
euclidean_dist = distance.euclidean(pelicula_A, pelicula_B)
euclidean_dist

0.8660254037844386

In [ ]:
pelicula_C = np.array([0, 0, 0.5])

# Calculamos la distancia Euclidiana
euclidean_dist = distance.euclidean(pelicula_A, pelicula_C)
euclidean_dist

5.522680508593631

In [ ]:
# Calculamos la distancia Euclidiana
euclidean_dist = distance.euclidean(pelicula_A, pelicula_B)
euclidean_dist

0.8660254037844386

La distancia Euclidiana entre la película A y B nos puede ayudar a determinar qué tan similares son en términos de preferencia de género, lo cual puede ser útil para recomendar películas a los usuarios basándose en sus calificaciones anteriores.

#### Métodos de Enlace (Linkage)

Una vez definida la medida de similitud, se deben considerar métodos para determinar la distancia entre clusters. Definimos entonces **$C_i$** y **$C_j$** dos **clusters** o grupos:

- **Enlace Simple**: La distancia entre dos clusters es igual a la distancia más corta de cualquier par de puntos en los clusters:
  $$
  D(C_i, C_j) = \min\{d(c_i, c_j): c_i \in C_i, c_j \in C_j\}
  $$


- **Enlace Completo**: La distancia entre dos clusters es igual a la distancia más larga de cualquier par de puntos en los clusters:
  $$
  D(C_i, C_j) = \max\{d(c_i, c_j): c_i \in C_i, c_j \in C_j\}
  $$

- **Enlace Promedio**: La distancia entre dos clusters es el promedio de las distancias entre todos los pares de puntos en los clusters:
  $$
  D(C_i, C_j) = \frac{1}{|C_i||C_j|}\sum_{c_i \in C_i}\sum_{c_j \in C_j} d(c_i, c_j)
  $$

- **Enlace del Centroide**: La distancia entre dos clusters se mide como la distancia entre los centroides de los clusters:
  $$
  D(C_i, C_j) = d(\text{centroide}(C_i), \text{centroide}(C_j))
  $$
  donde el centroide de un cluster $C$ con puntos $c_1, c_2, ..., c_m$ es $ \text{centroide}(C) = \frac{1}{m}\sum_{i=1}^{m} c_i $.

- **Enlace de Ward**: Este enfoque busca minimizar la varianza total dentro del cluster. La distancia entre dos clusters se calcula de tal manera que la varianza dentro de los clusters aumenta lo menos posible después de la fusión. La distancia entre dos clusters $A$ y $B$ según el criterio de Ward se puede calcular como:

  $$
  \Delta E(i,j) = \frac{|C_i||C_j|}{|C_i| + |C_j|} \cdot ||\mathbf{\mu}_i - \mathbf{\mu}_j||^2
  $$

    donde:
    
     - **$\mathbf{\mu}_i$ y $\mathbf{\mu}_j$** son los **vectores de medias** de los grupos $C_i$ y $C_j$ respectivamente.

<div style="text-align:center">
  <img src="../docs/figures/linkage-vs.png" alt="enlaces">
</div>

#### Algoritmo de Conglomerado Jerárquico

1. Comenzar tratando cada punto de datos como un cluster.
2. Calcular la matriz de distancias entre todos los pares de clusters. Tomemos como ejemplo:

      $$
      \begin{array}{cccc}
      & 1 & 2 & 3 & 4 & 5 & \\
      1 & 0 & 9 & 3 & 6 & 11 & \\
      2 & 9 & 0 & 7 & 5 & 10 & \\
      3 & 3 & 7 & 0 & 9 & 2 & \\
      4 & 6 & 5 & 9 & 0 & 8 & \\
      5 & 11 & 10 & 2 & 8 & 0 & \\
      \end{array}
      $$

3. Fusionar los dos clusters más cercanos basándose en el método de enlace seleccionado.

      Las observaciones que se unen inicialmente, son $(3, 5)$, que se unen a altura $2$. Una vez que se obtiene este clúster, hay que calcular su distancia al resto de los elementos, utilizando el enlace promedio por ejemplo:   

      $$
      \begin{align*}
      d\{\left(3,5\right), 1\} = \frac{1}{2} \left[d\{\left(3,1\right)\} + d\{\left(5,1\right)\}\right] = \frac{1}{2}\left(3 + 11\right) = 7 \\
      d\{\left(3,5\right), 2\} = \frac{1}{2} \left[d\{\left(3,2\right)\} + d\{\left(5,2\right)\}\right] = \frac{1}{2}\left(7 + 10\right) = 8.5 \\
      d\{\left(3,5\right), 4\} = \frac{1}{2} \left[d\{\left(3,4\right)\} + d\{\left(5,4\right)\}\right] = \frac{1}{2}\left(9 + 8\right) = 8.5
      \end{align*}
      $$

      Y la correspondiente matriz queda:
      
      $$
      \begin{array}{cccc}
      & (3,5) & 1 & 2 & 4 \\
      (3,5) & 0 & 7 & 8.5 & 8.5 \\
      1 & 7 & 0 & 9 & 6 \\
      2 & 8.5 & 9 & 0 & 5 \\
      4 & 8.5 & 6 & 5 & 0 \\
      \end{array}
      $$

      La distancia mínima en esta matriz corresponde a las observaciones $(2, 4)$, que forman un nuevo grupo, que se une a altura $=5$. Nuevamente debemos calcular la distancia entre estos clusters mediante la liga promedio.

      $$
      \begin{align*}
      d\left(\{3,5\}, \{2,4\}\right) = \frac{1}{4}\left[d\left(\{3,2\}\right) + d\left(\{5,2\}\right) + d\left(\{3,4\}\right) + d\left(\{5,4\}\right)\right] = \frac{1}{4}(7 + 9 + 10 + 8) = 8.5 \\
      d\left(\{3,5\}, 1\right) = \frac{1}{2}\left[d\left(\{3,1\}\right) + d\left(\{5,1\}\right)\right] = \frac{1}{2}(3 + 11) = 7 \\
      d\left(\{2,4\}, 1\right) = \frac{1}{2}\left[d\left(\{2,1\}\right) + d\left(\{4,1\}\right)\right] = \frac{1}{2}(9 + 6) = 7.5
      \end{align*}
      $$

      Que genera la matriz de distancias:

      $$
      \begin{array}{cccc}
      & (3,5) & 1 & (2,4) \\
      (3,5) & 0 & 7 & 8.5 \\
      1 & 7 & 0 & 7.5 \\
      (2,4) & 8.5 & 7.5 & 0 \\
      \end{array}
      $$

4. Actualizar la matriz de distancias para reflejar la distancia entre el nuevo cluster y los clusters existentes.
5. Repetir los pasos 3 y 4 hasta que todos los puntos de datos estén en un solo cluster.

      Cuya distancia mínima es $7$ y corresponde a la unión de los grupos $(3, 5)$ y $1$, que originan el grupo $(1, 3, 5)$. La distancia entre estos grupos es:

      $$
      \begin{align*}
      d(\{1,3,5\}, \{2,4\}) &= \frac{1}{6} [d(1,2) + d(3,2) + d(5,2) + d(1,4) + d(3,4) + d(5,4)] \\
      &= \frac{1}{6} [9 + 7 + 9 + 6 + 10 + 8] \\
      &= 8.166
      \end{align*}
      $$
      
      Que obtiene la matriz:

      $$
      \begin{array}{cccc}
      & (1,3,5) & (2,4) \\
      (1,3,5) & 0 & 8.166 \\
      (2,4) & 8.166 & 0 \\
      \end{array}
      $$

      Entonces, la distancia final a la que se unen todos los grupos es 8.166. Observe que los grupos y las distancias a las que se unen, coinciden con la gráfica correspondiente a esta distancia promedio.
      
6. Representación Visual: Dendrograma

#### Aplicaciónes y Limitaciones

<p style="font-size:25px;">Aplicaciones</p>

1. Biología Computacional y Genómica
2. Análisis de Redes Sociales
3. Segmentación de Clientes
4. Análisis de Documentos y Textos


<p style="font-size:25px;">Ventajas</p>

1. No requiere la especificación del número de clusters
2. Facilita la interpretación mediante dendrogramas
3. Detecta clusters con formas no esféricas
4. Útil para datos a pequeña escala y análisis exploratorio
5. Sensible a la estructura de los datos
6. Robusto a variaciones en los datos


<p style="font-size:25px;">Limitaciones</p>

1. Complejidad Computacional
2. Difícil de Escalar para Grandes Conjuntos de Datos
3. Sensibilidad a Outliers
4. Decisión de Corte del Dendrograma
5. Elección del Método de Enlace

### Ejemplo Práctico

#### Preparación de los Datos

**Objetivo**

El objetivo es categorizar los países utilizando factores socioeconómicos y de salud que determinen el desarrollo general del país.

**Acerca de la organización**

**HELP International** es una ONG humanitaria internacional comprometida con la lucha contra la pobreza y con proporcionar a las personas de países atrasados las comodidades básicas y alivio durante desastres naturales y calamidades.

**Planteamiento del problema**

HELP International ha logrado recaudar alrededor de **$10 millones**. Ahora, el CEO de la ONG necesita decidir cómo utilizar este dinero de manera estratégica y efectiva. Por lo tanto, el CEO debe tomar la decisión de elegir los países que necesitan más urgentemente ayuda.

In [ ]:
# Carga y diccionario del conjunto de datos
df, dc = pd.read_pickle('../data/help_ong.pkl')

In [ ]:
pd.set_option('display.max_colwidth', None)
dc

,Column Name,Description
0,country,Name of the country
1,child_mort,Death of children under 5 years of age per 1000 live births
2,exports,Exports of goods and services per capita. Given as %age of the GDP per capita
3,health,Total health spending per capita. Given as %age of GDP per capita
4,imports,Imports of goods and services per capita. Given as %age of the GDP per capita
5,Income,Net income per person
6,Inflation,The measurement of the annual growth rate of the Total GDP
7,life_expec,The average number of years a new born child would live if the current mortality patterns are to remain the same
8,total_fer,The number of children that would be born to each woman if the current age-fertility rates remain the same.
9,gdpp,The GDP per capita. Calculated as the Total GDP divided by the total population.


In [ ]:
df = df.set_index(['country'])

In [ ]:
df.sample(10)

,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp
country,,,,,,,,,
Yemen,56.3,30.0,5.18,34.4,4480,23.600,67.5,4.67,1310
Indonesia,33.3,24.3,2.61,22.4,8430,15.300,69.9,2.48,3110
Mauritania,97.4,50.7,4.41,61.2,3320,18.900,68.2,4.98,1200
Malawi,90.5,22.8,6.59,34.9,1030,12.100,53.1,5.31,459
Belgium,4.5,76.4,10.70,74.7,41100,1.880,80.0,1.86,44400
Serbia,7.6,32.9,10.40,47.9,12700,5.880,74.7,1.40,5410
"Congo, Rep.",63.9,85.1,2.46,54.7,5190,20.700,60.4,4.95,2740
Chile,8.7,37.7,7.96,31.3,19400,8.960,79.1,1.88,12900
Armenia,18.1,20.8,4.40,45.3,6700,7.770,73.3,1.69,3220


In [ ]:
df.describe()

,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp
count,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000
mean,38.270060,41.108976,6.815689,46.890215,17144.688623,7.781832,70.555689,2.947964,12964.155689
std,40.328931,27.412010,2.746837,24.209589,19278.067698,10.570704,8.893172,1.513848,18328.704809
min,2.600000,0.109000,1.810000,0.065900,609.000000,-4.210000,32.100000,1.150000,231.000000
25%,8.250000,23.800000,4.920000,30.200000,3355.000000,1.810000,65.300000,1.795000,1330.000000
50%,19.300000,35.000000,6.320000,43.300000,9960.000000,5.390000,73.100000,2.410000,4660.000000
75%,62.100000,51.350000,8.600000,58.750000,22800.000000,10.750000,76.800000,3.880000,14050.000000
max,208.000000,200.000000,17.900000,174.000000,125000.000000,104.000000,82.800000,7.490000,105000.000000


Dado la diferencia en escala estandarizamos nuestros datos:

In [ ]:
scaler = StandardScaler()

# Seleccionar las columnas a estandarizar
X = scaler.fit_transform(df)
X.mean().round(), X.std().round()

(child_mort   -0.0
 exports       0.0
 health        0.0
 imports       0.0
 income       -0.0
 inflation    -0.0
 life_expec    0.0
 total_fer     0.0
 gdpp          0.0
 dtype: float64,
 child_mort    1.0
 exports       1.0
 health        1.0
 imports       1.0
 income        1.0
 inflation     1.0
 life_expec    1.0
 total_fer     1.0
 gdpp          1.0
 dtype: float64)

#### Implementación del Método

Principales parámetros de Clustering Jerárquico Aglomerativo:
```python
AgglomerativeClustering(
    n_clusters=2,
    metric='euclidean',
    linkage='ward',
    distance_threshold=None,
)

#### Visualización e Implementación de Resultados

In [ ]:
# Utilizaremos el linkage 'ward' por defecto para el análisis preliminar
linked = linkage(X, method='ward')

# Calcular el coeficiente de silueta para diferentes números de clusters
silhouette_scores = []
range_n_clusters = range(3, 10)  # Exploraremos de 3 a 10 clusters

for n_clusters in range_n_clusters:
    clusterer = AgglomerativeClustering(n_clusters=n_clusters, metric='euclidean', linkage='ward')
    cluster_labels = clusterer.fit_predict(X)
    silhouette_avg = silhouette_score(X, cluster_labels)
    silhouette_scores.append(silhouette_avg)

In [ ]:
fig = ff.create_dendrogram(X,
                           labels=X.index,
                           linkagefun=lambda x: linkage(x, method='ward'))
fig.update_layout(width=1200, height=1000)
fig.show()

In [ ]:
score_plot(scores=silhouette_scores, range_n_clusters=range_n_clusters,
           operator='max', name_index = 'Índice de Silueta')

Basado en el análisis previo, elegimos un número de clusters (esta interpretación es meramente subjetiva y depende de lo que buscamos):

In [ ]:
# Eligiendo un número razonable basado en el análisis del coeficiente de silueta y el dendrograma (3 y 4 podríamos probar)
n_clusters_optimo = 3

# Realizar el clustering con el número óptimo de clusters
clusterer_final = AgglomerativeClustering(n_clusters=n_clusters_optimo,
                                          metric='euclidean',
                                          linkage='ward')

cluster_labels = clusterer_final.fit_predict(X)

In [ ]:
cluster_labels

array([2, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 2, 1, 1, 1, 1,
       1, 0, 1, 2, 2, 1, 2, 0, 1, 2, 2, 1, 1, 1, 2, 2, 1, 1, 2, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 2, 1, 0, 1, 0, 1, 1, 2, 2, 1,
       2, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 2, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 2, 2, 1, 1, 2, 0, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1,
       0, 0, 2, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 2, 1, 0, 2, 1, 1,
       2, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 2, 1, 1, 2, 1, 1, 1,
       1, 2, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2], dtype=int64)

In [ ]:
# Asignar las etiquetas de cluster
df['Grupos'] = cluster_labels

In [ ]:
df

,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp,Grupos
country,,,,,,,,,,
Afghanistan,90.2,10.0,7.58,44.9,1610,9.44,56.2,5.82,553,2
Albania,16.6,28.0,6.55,48.6,9930,4.49,76.3,1.65,4090,1
Algeria,27.3,38.4,4.17,31.4,12900,16.10,76.5,2.89,4460,1
Angola,119.0,62.3,2.85,42.9,5900,22.40,60.1,6.16,3530,1
Antigua and Barbuda,10.3,45.5,6.03,58.9,19100,1.44,76.8,2.13,12200,1
...,...,...,...,...,...,...,...,...,...,...
Vanuatu,29.2,46.6,5.25,52.7,2950,2.62,63.0,3.50,2970,1
Venezuela,17.1,28.5,4.91,17.6,16500,45.90,75.4,2.47,13500,1
Vietnam,23.3,72.0,6.84,80.2,4490,12.10,73.1,1.95,1310,1


In [ ]:
pd.DataFrame(df.groupby('Grupos')['Grupos'].count()).rename({'Grupos':'Paises'},axis=1)

,Paises
Grupos,
0,34
1,106
2,27


In [ ]:
df.groupby('Grupos').median()

,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp
Grupos,,,,,,,,,
0,4.5,50.05,9.485,37.25,41250.0,1.670,80.4,1.870,41850.0
1,20.5,37.30,6.015,49.25,9890.0,6.045,72.3,2.395,4520.0
2,101.0,22.20,5.690,39.20,1430.0,5.450,57.7,5.340,575.0


Grupo de Necesidades Urgentes:

In [ ]:
df[df.Grupos == 2].index

Index(['Afghanistan', 'Benin', 'Burkina Faso', 'Burundi', 'Cameroon',
       'Central African Republic', 'Chad', 'Comoros', 'Congo, Dem. Rep.',
       'Cote d'Ivoire', 'Gambia', 'Guinea', 'Guinea-Bissau', 'Haiti', 'Kenya',
       'Madagascar', 'Malawi', 'Mali', 'Mozambique', 'Niger', 'Rwanda',
       'Senegal', 'Sierra Leone', 'Tanzania', 'Togo', 'Uganda', 'Zambia'],
      dtype='object', name='country')

Grupo con potencial de desarrollo a mediano plazo:

In [ ]:
df[df.Grupos == 1].index

Index(['Albania', 'Algeria', 'Angola', 'Antigua and Barbuda', 'Argentina',
       'Armenia', 'Azerbaijan', 'Bahamas', 'Bangladesh', 'Barbados',
       ...
       'Tunisia', 'Turkey', 'Turkmenistan', 'Ukraine', 'Uruguay', 'Uzbekistan',
       'Vanuatu', 'Venezuela', 'Vietnam', 'Yemen'],
      dtype='object', name='country', length=106)

Grupo Estable:

In [ ]:
df[df.Grupos == 0].index

Index(['Australia', 'Austria', 'Bahrain', 'Belgium', 'Brunei', 'Canada',
       'Denmark', 'Finland', 'France', 'Germany', 'Greece', 'Iceland',
       'Ireland', 'Israel', 'Italy', 'Japan', 'Kuwait', 'Libya', 'Luxembourg',
       'Malta', 'Netherlands', 'New Zealand', 'Norway', 'Oman', 'Portugal',
       'Qatar', 'Saudi Arabia', 'Singapore', 'Spain', 'Sweden', 'Switzerland',
       'United Arab Emirates', 'United Kingdom', 'United States'],
      dtype='object', name='country')

Mapa Ilustrativo:

In [ ]:
dfi = df.reset_index()[['country','Grupos']]

dfi.Grupos[dfi['Grupos'] == 0] = 'Estables'
dfi.Grupos[dfi['Grupos'] == 1] = 'En Desarrollo'
dfi.Grupos[dfi['Grupos'] == 2] = 'Urgentes'

fig = px.choropleth(dfi,
                    locationmode='country names',
                    locations='country',
                    color='Grupos',
                    color_discrete_map={'Estables': 'darkgreen', 'En Desarrollo': '#DAA520', 'Urgentes': 'darkred'})


fig.update_layout(
    title_text='Mapa Global de Importancia para HELP ORG',
    geo=dict(showframe=False, showcoastlines=False, projection_type='equirectangular')
)

fig.show()